**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 4 - Open Science Platform - Cross link User and Project accounts

1 - Load required modules

In [1]:
from new_helper import *
from super_helper import *
from loguru import logger

2 - Set variables and values

In [2]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 3

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts, user_account_short_ids  = read_user_accounts_from_jsonld('logs/accounts.json')
project_accounts, project_account_short_ids = read_project_accounts_from_jsonld('logs/projects.json')

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [3]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-11 11:55:28,977 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-11 11:55:28.977 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` (see item 2) with a entry number related to the position of an existing object in `datasets/accounts.json` and `datasets/projects.json`. Example: `json_ld_index = 2`, selects the second object in each json file.

6 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [4]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)



# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
user_account_short_id = user_account_short_ids[json_ld_index]
project_account = project_accounts[json_ld_index]
project_account_short_id = project_account_short_ids[json_ld_index]

logger.debug(f"{user_account['account_id']}")
logger.debug(f"{user_account_short_id['account_id']}")

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-02-11 11:55:28.982 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-11 11:55:28.988 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-11 11:55:28.988 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-11 11:55:28.988 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-11 11:55:28.989 | DEBUG    | __main__:<module>:11 - upbeat_neumann@test
2025-02-11 11:55:28.989 | DEBUG    | __main__:<module>:12 - upbeat_neumann
2025-02-11 11:55:28.989 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-11 11:55:28,995 - INFO - None
2025-02-11 11:55:28,995 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-11 11:55:28,996 - INFO - ('ENOUGH_SIGNATURES_COLLECTED

2025-02-10 15:06:28,131 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:06:28,139 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:06:28.141 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-10 15:06:28.161 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-10 15:06:28.162 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-10 15:06:28.164 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-10 15:06:28.166 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "set_account_detail"
2025-02-10 15:06:28.167 | DEBUG    | integration_helpers

7 - Load the sart contract hash

In [5]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)

2025-02-11 11:55:33,826 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-11 11:55:33.827 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-11 11:55:33.828 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-11 11:55:33.847 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-11 11:55:33.848 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-11 11:55:33.849 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"


8 - Queries Iroha 1 for the User account and exposes the variable `project_id` from user details.

In [6]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id['account_id'], "test")
address = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
# account_metadata_cid = process_raw_data(address) 

# Process the raw data
account_metadata_cid, project_id = process_raw_data(address) #TO DO - From now

2025-02-11 11:55:33.858 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-11 11:55:33,860 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-11 11:55:33,875 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-11 11:55:33,876 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-11 11:55:36,796 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-11 11:55:36,805 - INFO - ('COMMITTED', 5, 0)
2025-02-11 11:55:36,807 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '8fc5456a1888152a9fff1ca7470d62d922d9391a3a01a5ac0503d5d645c1f07d', 'timestamp': 1739274936807}
2025-02-11 11:55:36.809 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-11 11:55:36,814 - INFO - Appended data under 'upbeat_neumann@test' in logs/account_data.json
2025-02-11 11:55:36.816 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-11 11:

9 - Queries Iroha 1 for the project account.

In [7]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, project_account_short_id['account_id'], "test")
details = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-11 11:55:36.869 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-11 11:55:36,870 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-11 11:55:36,886 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-11 11:55:36,888 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-11 11:55:39,837 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-11 11:55:39,846 - INFO - ('COMMITTED', 5, 0)
2025-02-11 11:55:39,848 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'e149f9c2678e47414c2eaef50aa08e4a0fb18c89370630fbde0549cce9e5b04c', 'timestamp': 1739274939848}
2025-02-11 11:55:39.850 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-11 11:55:39,854 - INFO - Appended data under '72144@test' in logs/account_data.json
2025-02-11 11:55:39.855 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-11 11:55:39.857

10 - Saves `project_id` in the local file sytem for use in Notebook 4

In [8]:
# Dumping the variable to JSON
dump_variable(project_id, "Project Id")

2025-02-11 11:55:39.894 | DEBUG    | integration_helpers:tracer:35 - 	Entering "dump_variable"
2025-02-11 11:55:39,898 - INFO - Variable 'Project Id' successfully dumped to temp/Project Id.json
2025-02-11 11:55:39.899 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "dump_variable"
